In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install https://github.com/pytorch/text/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 2.3 MB 5.5 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for torchtext
  Running setup.py clean for torchtext
Failed to build torchtext
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
  error: subprocess-exited-with-error
  
  × Running setup.py install for torchtext did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for torchtext ... error
  Rolling back uninsta

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
import random
import torch
import torchtext
from torchtext import data
from torchtext import datasets

seed = 42

torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

txt = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

labels = data.LabelField(dtype = torch.float)

In [ ]:
train_data, test_data = datasets.IMDB.splits(txt, labels)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 9.24MB/s]


In [ ]:
train_data, valid_data = train_data.split(random_state = random.seed(seed))

In [ ]:
num_words = 25_000

txt.build_vocab(train_data,
                 max_size = num_words,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

labels.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:18<00:00, 21709.35it/s]


In [ ]:
btch_size = 64

train_itr, valid_itr, test_itr = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = btch_size,
    sort_within_batch = True,
    device = device)

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, word_limit, dimension_embedding, dimension_hidden, dimension_output, num_layers,
                 bidirectional, dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(word_limit, dimension_embedding, padding_idx = pad_idx)

        self.rnn = nn.LSTM(dimension_embedding,
                           dimension_hidden,
                           num_layers=num_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(dimension_hidden * 2, dimension_output)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text, len_txt):


        embedded = self.dropout(self.embedding(text))


        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, len_txt.to('cpu'))

        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)


        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        return self.fc(hidden)


In [ ]:
dimension_input = len(txt.vocab)
dimension_embedding = 100
dimension_hddn = 256
dimension_out = 1
layers = 2
bidirectional = True
droupout = 0.5
idx_pad = txt.vocab.stoi[txt.pad_token]

model = RNN(dimension_input,
            dimension_embedding,
            dimension_hddn,
            dimension_out,
            layers,
            bidirectional,
            droupout,
            idx_pad)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [ ]:
pretrained_embeddings = txt.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[ 1.9269,  1.4873,  0.9007,  ...,  0.1233,  0.3499,  0.6173],
        [ 0.7262,  0.0912, -0.3891,  ...,  0.0821,  0.4440, -0.7240],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1014,  1.2206,  1.1687,  ..., -0.6221,  0.2006,  0.7183],
        [-0.6763,  0.3646,  0.3423,  ..., -0.4848,  0.3119, -0.4102],
        [-0.5443,  0.2484,  0.1103,  ..., -0.2516, -0.5471,  0.6822]])

In [ ]:
unique_id = txt.vocab.stoi[txt.unk_token]

model.embedding.weight.data[unique_id] = torch.zeros(dimension_embedding)
model.embedding.weight.data[idx_pad] = torch.zeros(dimension_embedding)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1014,  1.2206,  1.1687,  ..., -0.6221,  0.2006,  0.7183],
        [-0.6763,  0.3646,  0.3423,  ..., -0.4848,  0.3119, -0.4102],
        [-0.5443,  0.2484,  0.1103,  ..., -0.2516, -0.5471,  0.6822]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def bin_acc(preds, y):

    predictions = torch.round(torch.sigmoid(preds))
    correct = (predictions == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, itr, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for i in itr:

        optimizer.zero_grad()

        text, len_txt = i.text

        predictions = model(text, len_txt).squeeze(1)

        loss = criterion(predictions, i.label)

        acc = bin_acc(predictions, i.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(itr), epoch_acc / len(itr)

In [ ]:
def evaluate(model, itr, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for i in itr:

            text, len_txt = i.text

            predictions = model(text, len_txt).squeeze(1)

            loss = criterion(predictions, i.label)

            acc = bin_acc(predictions, i.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(itr), epoch_acc / len(itr)

In [ ]:
import time

def epoch_time(start_time, end_time):
    used_time = end_time - start_time
    used_mins = int(used_time / 60)
    used_secs = int(used_time - (used_mins * 60))
    return used_mins, used_secs

In [ ]:
num_epochs = 5

best_valid_loss = float('inf')

for epoch in range(num_epochs):

    start_time = time.time()

    train_loss, train_acc = train(model, train_itr, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_itr, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 34s
	Train Loss: 0.661 | Train Acc: 58.52%
	 Val. Loss: 0.644 |  Val. Acc: 63.70%
Epoch: 02 | Epoch Time: 0m 34s
	Train Loss: 0.560 | Train Acc: 71.99%
	 Val. Loss: 0.383 |  Val. Acc: 83.48%
Epoch: 03 | Epoch Time: 0m 35s
	Train Loss: 0.401 | Train Acc: 82.65%
	 Val. Loss: 0.315 |  Val. Acc: 87.07%
Epoch: 04 | Epoch Time: 0m 34s
	Train Loss: 0.309 | Train Acc: 87.42%
	 Val. Loss: 0.283 |  Val. Acc: 88.69%
Epoch: 05 | Epoch Time: 0m 35s
	Train Loss: 0.255 | Train Acc: 90.06%
	 Val. Loss: 0.266 |  Val. Acc: 89.36%


In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_itr, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.291 | Test Acc: 88.18%


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def pred(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [txt.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
sent=["positive","neutral","negative"]
def print_sent(x):
  if (x<0.3): print(sent[0])
  elif (x>0.3 and x<0.7): print(sent[1])
  else: print(sent[2])

In [ ]:
print_sent(pred(model, "This film was average"))


neutral


In [ ]:
print_sent(pred(model, "This film is horrible"))

negative


In [ ]:
print_sent(pred(model, "This film was great"))

positive


In [ ]:
print_sent(pred(model, "This was the best movie i have seen in a while. The cast was great and the script was awesome, and the direction just blew my mind"))

positive


In [ ]:
print_sent(pred(model, "the cast was dumb"))

negative


In [ ]:
print_sent(pred(model,"Why does this fil exist"))

negative
negative
